# Project

## **주제와 목적**
최근 북미, 유럽, 일본, 그외 지역에서의 게임 판매량 데이터를 분석하고  
분석을 바탕으로 다음 분기 어떤 게임을 설계 할 것인지 정하는것이다    

## **게임을 설계하기 위해서 무엇을 고려해야할까?**
게임을 설계하는 방향은 "판매량이 많은 게임" 을 만드는 것이다  
그러므로 게임의 판매량에 영향을 주는 요인이 무엇인지 확인하고  
그 요인들을 바탕으로 게임의 장르, 플랫폼을 선택해야한다.

### **1. 장르는 어떻게 선택할까?**

우선 장르와 지역, 연도의 관련성을 분석해보자
1. 지역별 장르의 선호도가 다를까?
2. 연도별 장르의 트랜드가 있는가?

이 과정을 통해 장르와 연관성 있는것을 찾아낸다  

### **2. 플랫폼은 어떻게 선택할까?**

플랫폼과 판매량의 관련성을 분석해보자
1. 플랫폼별 선호도가 다를까?


### **3. 그 후 설계 방향은?**
위의 연관성 있는 기준들을 판매량이 많은 게임들에 적용시켜  
설계할 게임의 최적의 장르와 플렛폼을 고를 수 있다


분석을 시작하기전에, 데이터를 좀더 깔끔하게 다듬는 전처리 작업을 진행하였다  
# 전처리작업(EDA)
우선 데이터내의 수치를 일관성있게 통일하기위해 전처리작업을 진행하였다  
전처리작업이란, 숫자의단위나 알파벳의 대소문자의 통일, 알수없는값이나 비어있는값을 처리하여 데이터의 오류를 줄이는 과정이다.

In [284]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [285]:
import pandas as pd
import numpy as np
from scipy.stats import chi2_contingency, f_oneway
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler


# 데이터 불러오기
df = pd.read_csv('/content/drive/MyDrive/AI_bootcamp/vgames2.csv', encoding='CP949')

# 인덱스제거
df = df.drop(['Unnamed: 0','Name','Publisher'], axis=1)

In [286]:
# 데이터 전처리하기


# 판매량을 (0.00) 단위로 통일하고 float으로 변경하는 함수 선언
def unification(string):
    # K가 있다면 지우고 천 곱하고 백만으로 나누기
    if ('K' in string) == True:
        # 만약 0K 라면 0.001로 바로바꾸기
        if string == '0K':
            return float(0.001)
        else:
            remove_k = string.replace('K','')
            return float(remove_k)*0.001
    # M이 있다면 지우기
    elif ('M' in string) == True:
        # 만약 OM 이라면 1.0으로 바꾸기
        if string == '0M':
            return float(1.0)
        else:
            remove_m = string.replace('M','')
            return float(remove_m)
    # 둘다없다면 타입만바꾸기
    else:
        return float(string)

# 판매량 칼럽의 단위통일시키기
df['NA_Sales'] = df['NA_Sales'].apply(unification)
df['JP_Sales'] = df['JP_Sales'].apply(unification)
df['EU_Sales'] = df['EU_Sales'].apply(unification)
df['Other_Sales'] = df['Other_Sales'].apply(unification)

# datatype이 object->float으로 되었는지 확인
df.dtypes

Platform        object
Year           float64
Genre           object
NA_Sales       float64
EU_Sales       float64
JP_Sales       float64
Other_Sales    float64
dtype: object

전처리가 끝난 데이터로 분석해보자  


# 1. 장르는 어떻게 선택할까?

- 지역별 장르의 선호도는 다를까?  
  - 장르들의 지역별 판매량이 비슷하다면  ->  선호도가 비슷하다  
  - 장르들의 지역별 판매량이 다르다면  ->  선호도가 다르다  


In [287]:
# 장르와 판매량만 포함하는 df2 선언후 결측치 처리
df2 = df
df2 = df2.drop(['Year','Platform'],axis=1)
df2 = df2.dropna()
df2.isna().sum()
df2.head()

,Genre,NA_Sales,EU_Sales,JP_Sales,Other_Sales
0,Action,0.04,0.00,0.00,0.00
1,Action,0.17,0.00,0.00,0.01
2,Adventure,0.00,0.00,0.02,0.00
3,Misc,0.04,0.00,0.00,0.00
4,Platform,0.12,0.09,0.00,0.04


In [289]:
# 장르 선호도 비교해보기

# 장르들 목록 선언
genres = ['Action', 'Adventure', 'Fighting', 'Misc', 'Platform', 'Puzzle',
       'Racing', 'Role-Playing', 'Shooter', 'Simulation', 'Sports',
       'Strategy']

# 각 장르에대한 지역별 판매량 비교
for genre in genres:
    genre_data = df2[df2['Genre'] == genre]
    result = f_oneway(genre_data['NA_Sales'],genre_data['JP_Sales'],genre_data['EU_Sales'],genre_data['Other_Sales'])
    print(genre, '/p-value =', result[1])

# 여기서의 " 귀무가설 = 지역간 장르별 판매량은 차이가없다 "
# 하지만 장르들의 p-value 값이 모두 0.05보다 현저히 작으므로 귀무가설을 기각할수 있다
# 즉 "각 지역간의 장르별 판매량은 차이가 있다"
# 다시말해 지역별 선호도는 다르다

Action /p-value = 9.917382232427142e-147
Adventure /p-value = 2.0945837982584205e-20
Fighting /p-value = 1.2330556384385163e-43
Misc /p-value = 6.440700553325507e-43
Platform /p-value = 3.476396668290617e-28
Puzzle /p-value = 7.747629545653912e-07
Racing /p-value = 5.541020506203407e-39
Role-Playing /p-value = 2.1695107684387342e-27
Shooter /p-value = 1.8083226259160457e-63
Simulation /p-value = 1.0172201539635495e-21
Sports /p-value = 1.4237698139211029e-39
Strategy /p-value = 2.6194762348013894e-12


- 연도별 게임의 트랜드가 있을까?
  - 연도의 장르별 갯수가 다르다면 - > 트랜드가 있다
  - 연도의 장르별 갯수가 비슷하다면 - > 트랜드가 없다


In [302]:
# 연도와 장르만 포함한 df3 선언후 결측치 처리
df3 = df[['Year','Genre']].dropna()
df3.isna().sum()

# 네자리로 통일시킨후 catogorical 하게 바꾸는 함수 선언
def make_year(year):
    if year < 21:
        return str(year+2000)
    elif year > 80 and year <= 99:
        return str(year+1900)
    else:
        return str(year)

# apply 사용해서 단위 통일
df3['Year'] = df3['Year'].apply(make_year)
df3.head()

,Year,Genre
0,2008.0,Action
1,2009.0,Action
2,2010.0,Adventure
3,2010.0,Misc
4,2010.0,Platform


In [312]:
# 연도별 장르의 갯수를 나타내는 데이터프레임 선언
year_genre = pd.crosstab(df3.Year,df3.Genre)
year_genre.head()

Genre,Action,Adventure,Fighting,Misc,Platform,Puzzle,Racing,Role-Playing,Shooter,Simulation,Sports,Strategy
Year,,,,,,,,,,,,
1980.0,1,0,1,4,0,0,0,0,2,0,0,0
1981.0,25,0,0,0,3,2,1,0,10,1,4,0
1982.0,18,0,0,1,5,3,2,0,5,0,2,0
1983.0,7,1,0,1,5,1,0,0,1,0,1,0
1984.0,1,0,0,1,1,3,3,0,3,0,2,0


In [325]:
# 연도별 데이터나누기

# 연도리스트 생성
years = ['1980.0', '1981.0', '1982.0', '1983.0', '1984.0', '1985.0', '1986.0',
       '1987.0', '1988.0', '1989.0', '1990.0', '1991.0', '1992.0', '1993.0',
       '1994.0', '1995.0', '1996.0', '1997.0', '1998.0', '1999.0', '2000.0',
       '2001.0', '2002.0', '2003.0', '2004.0', '2005.0', '2006.0', '2007.0',
       '2008.0', '2009.0', '2010.0', '2011.0', '2012.0', '2013.0', '2014.0',
       '2015.0', '2016.0', '2017.0', '2020.0']

# 연도별 데이터를 df_연도 형식의 데이터프레임으로 분류
for year in years:
    df_year = df3[df['Year'] == year]

dd = df_1980.drop('Year',axis=1)
dd['Total_Sales'] = dd['NA_Sales']+dd['EU_Sales']+dd['JP_Sales']+dd['Other_Sales']

dd = dd[['Genre','Total_Sales']]

dd
# ddf = dd.groupby('Genre').mean()
# ddf
# stats.ttest_1samp(dff['Total_Sales'], .5)


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:13: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  del sys.path[0]


,Genre,Total_Sales
5723,Shooter,4.310
5938,Misc,1.150
7317,Misc,0.231
9226,Misc,0.270
10121,Misc,1.050
13133,Action,0.340
14264,Fighting,0.770
15012,Shooter,2.760


# 2. 플랫폼은 어떻게 선택할까?

- 지역별 플랫폼의 선호도가 다를까?
- 연도별 플랫폼의 트랜드가 있을까?



In [345]:
# 총매출을 나타내는 Total_Sales 열 추가
df['Total_Sales'] = df['EU_Sales'] + df['JP_Sales'] + df['NA_Sales'] + df['Other_Sales']

# 플랫폼과 총판매액만 나타내는 데이터프레임 선언, 결측치제거
df_platform_genre = df[['Platform','Genre','Total_Sales']].dropna()
df_platform_genre.head()



,Platform,Genre,Total_Sales
0,DS,Action,0.04
1,Wii,Action,0.18
2,PSP,Adventure,0.02
3,DS,Misc,0.04
4,PS3,Platform,0.25


In [344]:
# 플랫폼별 장르의 판매량 나타내는 함수 선언
platforms = ['2600', '3DO', '3DS', 'DC', 'DS', 'GB', 'GBA', 'GC', 'GEN', 'GG', 'N64',
       'NES', 'NG', 'PC', 'PCFX', 'PS', 'PS2', 'PS3', 'PS4', 'PSP', 'PSV',
       'SAT', 'SCD', 'SNES', 'TG16', 'WS', 'Wii', 'WiiU', 'X360', 'XB',
       'XOne']

def platformGenre(df):
    df_empty = pd.DataFrame(columns=['Action','Adventure',	'Fighting',	'Misc',	'Platform',	'Puzzle',	'Racing',	'Role-Playing',	'Shooter',	'Simulation',	'Sports',	'Strategy'])
    for platform in platforms:
        plat = df[df['Platform'] == platform]
        plat = plat[['Genre','Total_Sales']]
        result = plat.groupby('Genre').mean().T
        result = result.rename(index={'Total_Sales': platform})
        df_empty = pd.concat([df_empty,result])
    return df_empty

platformGenre(df_platform_genre)

,Action,Adventure,Fighting,Misc,Platform,Puzzle,Racing,Role-Playing,Shooter,Simulation,Sports,Strategy
2600,0.479525,0.850000,0.620000,0.714200,1.473333,1.335545,0.485000,NaN,1.144583,0.440000,0.268182,NaN
3DO,NaN,0.060000,NaN,NaN,NaN,0.020000,NaN,NaN,NaN,0.020000,NaN,NaN
3DS,0.318203,0.157297,0.745714,0.197566,1.151786,0.278000,1.319091,0.722000,0.185714,0.937333,0.238462,0.138000
DC,0.420000,0.227273,0.235833,NaN,1.265000,NaN,0.441667,0.170000,0.110000,0.520000,0.365100,NaN
DS,0.348050,0.214439,0.200000,0.371446,0.841217,0.362152,0.575985,0.638860,0.216690,0.470303,0.228122,0.219367
GB,1.321667,3.430000,NaN,1.668750,2.890000,3.164667,2.275000,4.201952,1.190000,0.710000,1.006667,1.151429
GBA,0.339299,0.390541,0.224348,0.337182,0.557831,0.312439,0.293906,0.918778,0.114000,0.330000,0.185466,0.414500
GC,0.393099,0.296500,0.438810,0.464444,0.405356,0.331667,0.363016,0.487407,0.283542,0.717500,0.230373,0.389091
GEN,1.246667,0.095000,1.178000,0.030000,2.205714,NaN,0.260000,0.090000,0.130000,NaN,1.070000,0.190000
GG,NaN,NaN,NaN,NaN,0.040000,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# 4. 출고량이 많은 게임을 분석하고 시각화해보자

출고량 순서대로 그래프를 그려보자

# 결론
## 판매량과 다른 요소들의 관계를 비교해보자  
위의 분석들을 통해 다음과 같은 결론을 내릴 수 있다  

판매량 - 장르  
판매량 - 플랫폼  
판매량 - 

이 우선순위를 바탕으로  
최근 판매량이 많은 게임을 분석하여 적용시켜본다면  

목적에 부합하는  
"잘 팔리는 게임"을 만들 수 있는 가능성이 높다  

~ 한 게임을 만드는것이 좋다

In [ ]:
g1 = np.array([0, 31, 6, 26, 40])

g2 = np.array([24, 15, 12, 22, 5])

g3 = np.array([32, 52, 30, 18, 36])

pd.DataFrame(g1)

,0
0,0
1,31
2,6
3,26
4,40


# 잘못된 자료들

In [ ]:
# 정체불명의 장르분포비교 ,
ge_lo = df2.groupby('Genre').sum().T

# 각 지역별 장르의 분포를 비교하는 F-test 실시
ge_loc = f_oneway(ge_lo['Action'],ge_lo['Adventure'],ge_lo['Fighting'],ge_lo['Misc'],ge_lo['Platform'],ge_lo['Puzzle'],ge_lo['Racing'],ge_lo['Role-Playing'],ge_lo['Shooter'],ge_lo['Simulation'],ge_lo['Sports'],ge_lo['Strategy'])
print("P-value :", ge_loc.pvalue)

print("""
여기서의 귀무가설은 "장르별 판매량은 차이가없다" 이다

p-value의 값은 0.051 이므로 귀무가설을 기각할 수 없다
지역의 장르별 판매량은 차이가없다. 즉, 선호도가 비슷하다 
""")